In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet50
from torch.utils.data import DataLoader
from pycocotools.coco import COCO
from sklearn.model_selection import GridSearchCV
import numpy as np
import os

# Define Darknet-53 Backbone
class Darknet53(nn.Module):
    def __init__(self):
        super(Darknet53, self).__init__()
        self.resnet = resnet50(pretrained=True)  
        self.features = nn.Sequential(*list(self.resnet.children())[:-2])  # Remove FC layers

    def forward(self, x):
        return self.features(x)


class YOLOv3(nn.Module):
    def __init__(self, num_classes):
        super(YOLOv3, self).__init__()
        self.backbone = Darknet53()
        self.detect_head = nn.Conv2d(2048, num_classes, kernel_size=1)  # Simplified detection head

    def forward(self, x):
        x = self.backbone(x)
        x = self.detect_head(x)
        return x

# Data Preparation
def load_coco_data(data_dir, split='train'):
    """
    Load COCO dataset using pycocotools and prepare DataLoader
    """
    coco = COCO(os.path.join(data_dir, f'annotations/instances_{split}2017.json'))
    img_dir = os.path.join(data_dir, f'{split}2017')
    transform = transforms.Compose([
        transforms.Resize((416, 416)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    dataset = torchvision.datasets.CocoDetection(img_dir, coco, transform=transform)
    dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
    return dataloader

# Define Training and Evaluation Functions
def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for images, targets in dataloader:
        images = images.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, torch.zeros_like(outputs))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

def evaluate_model(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for images, targets in dataloader:
            images = images.to(device)
            outputs = model(images)
            loss = criterion(outputs, torch.zeros_like(outputs))
            total_loss += loss.item()
    return total_loss / len(dataloader)

# Hyperparameter Tuning with GridSearchCV
class TorchGridSearch:
    def __init__(self, param_grid, model_class, train_loader, val_loader, device):
        self.param_grid = param_grid
        self.model_class = model_class
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device

    def fit(self):
        best_params = None
        best_loss = float('inf')
        for params in self.param_grid:
            model = self.model_class(**params).to(self.device)
            optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
            criterion = nn.MSELoss()
            train_loss = train_one_epoch(model, self.train_loader, optimizer, criterion, self.device)
            val_loss = evaluate_model(model, self.val_loader, criterion, self.device)
            if val_loss < best_loss:
                best_loss = val_loss
                best_params = params
        return best_params, best_loss

# Main Execution
if __name__ == "__main__":
    # Parameters
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    data_dir = "/path/to/coco"  # Update with your COCO dataset path
    train_loader = load_coco_data(data_dir, split='train')
    val_loader = load_coco_data(data_dir, split='val')

    # Hyperparameter Grid
    param_grid = [
        {'num_classes': [80], 'learning_rate': [1e-3, 1e-4, 1e-5]}
    ]

    grid_search = TorchGridSearch(param_grid, YOLOv3, train_loader, val_loader, device)
    best_params, best_loss = grid_search.fit()

    print(f"Best Params: {best_params}")
    print(f"Validation Loss: {best_loss}")


In [ ]:
# Applying best features from

# Darknet-53 Backbone
class Darknet53(nn.Module):
    def __init__(self):
        super(Darknet53, self).__init__()
        self.resnet = torchvision.models.resnet50(pretrained=True)
        self.features = nn.Sequential(*list(self.resnet.children())[:-2])

    def forward(self, x):
        return self.features(x)

# YOLOv3 Architecture
class YOLOv3(nn.Module):
    def __init__(self, num_classes):
        super(YOLOv3, self).__init__()
        self.backbone = Darknet53()
        self.conv1 = nn.Conv2d(2048, 1024, kernel_size=1)
        self.conv2 = nn.Conv2d(1024, num_classes + 5, kernel_size=1)  # (num_classes + 5 for bbox coords)

    def forward(self, x):
        x = self.backbone(x)
        x = self.conv1(x)
        x = self.conv2(x)
        return x

# Data Preparation
def load_coco_data(data_dir, split='train'):
    coco = COCO(os.path.join(data_dir, f'annotations/instances_{split}2017.json'))
    img_dir = os.path.join(data_dir, f'{split}2017')
    transform = transforms.Compose([
        transforms.Resize((416, 416)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    dataset = torchvision.datasets.CocoDetection(img_dir, coco, transform=transform)
    dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
    return dataloader

# Define Loss Function (Simplified)
class YOLOLoss(nn.Module):
    def __init__(self, num_classes, anchors):
        super(YOLOLoss, self).__init__()
        self.num_classes = num_classes
        self.anchors = anchors

    def forward(self, predictions, targets):
        # Placeholder: Implement YOLO-specific loss functions
        return torch.tensor(0.0)

# Non-Max Suppression (NMS)
def apply_nms(predictions, confidence_threshold=0.5, iou_threshold=0.5):
    boxes, scores, classes = [], [], []
    for pred in predictions:
        mask = pred[:, 4] > confidence_threshold
        filtered_boxes = pred[mask]
        nms_indices = nms(filtered_boxes[:, :4], filtered_boxes[:, 4], iou_threshold)
        boxes.append(filtered_boxes[nms_indices])
    return torch.cat(boxes) if len(boxes) > 0 else torch.empty(0)

# Training Function
def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for images, targets in dataloader:
        images = images.to(device)
        optimizer.zero_grad()
        predictions = model(images)
        loss = criterion(predictions, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

# Main Execution
if __name__ == "__main__":
    # Hyperparameters from YOLOv3 Paper
    lr = 1e-3  # Learning rate
    momentum = 0.9  # Momentum for SGD
    weight_decay = 5e-4  # L2 regularization
    batch_size = 16  # Mini-batch size
    epochs = 50  # Number of training epochs

    # Load Data
    data_dir = "/path/to/coco"  # Update with COCO dataset path
    train_loader = load_coco_data(data_dir, split='train')
    val_loader = load_coco_data(data_dir, split='val')

    # Model, Optimizer, and Criterion
    num_classes = 80  # COCO dataset has 80 classes
    anchors = [[116, 90], [156, 198], [373, 326]]  # YOLOv3 default anchors
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = YOLOv3(num_classes).to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    criterion = YOLOLoss(num_classes, anchors)

    # Training Loop
    for epoch in range(epochs):
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {train_loss:.4f}")

    print("Training Complete!")
